In [ ]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
import json
import time
import paho.mqtt.client as mqtt

brokerAddress = '172.17.0.1'
port = 1883
topic = 'service/test/machine'

dataFrameSize=50

count = 0
x_start = 0
y_start = 0

Connected = False  # global variable for keeping state of connection

def bytesToJsonObject(payloadBytes):
    # Decode UTF-8 bytes to Unicode, and convert single quotes 
    # to double quotes to make it valid JSON
    payloadJson = payloadBytes.decode('utf8').replace("'", '"')
    jsonObject = json.loads(payloadJson)
    return jsonObject

def currentTimeMillis():
    return round(time.time() * 1000)

def publishRandomTimeSeriesData(client, topic):
    data_set = {"timestamp": str(currentTimeMillis()), "value": str(np.random.uniform(0,100)) }
    message = json.dumps(data_set)
    print(message)
    return client.publish(topic, json.dumps(data_set));

def on_connect(client, userdata, flags, rc):
    if rc == 0:
        print('Connected to broker')
        global Connected
        Connected = True
    else:
        print("Connection failed")

def on_message(client, userdata, msg):
    print(msg.topic+" "+str(msg.payload))
    # payload e.g, '{ "timestamp": "1629122452119000000", "value": "54.405711" }'

    # plot data
    global count
    count+=1
    time.sleep(0.1)
    data = bytesToJsonObject(msg.payload)
    y = float(data['value'])
    # https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.plot.html
    plt.pause(0.001)

    global x_start, y_start, dataFrameSize
    if x_start == 0:
        ax.plot(count, y, '.-k')
    else:
        ax.plot([x_start,count], [y_start,y], '.-k')
        ax.plot(count, y, 'o')
        if x_start > 20:
            plt.xlim([x_start - dataFrameSize, x_start])
   
    x_start = count
    y_start = y
    fig.canvas.draw()

# initialize chart
fig = plt.figure(figsize=(16,5), dpi= 100, facecolor='w', edgecolor='k')
ax = fig.add_subplot(111)
plt.ion()
plt.title('Data from mqtt broker')
fig.show()
fig.canvas.draw()

# set paho.mqtt callback
client = mqtt.Client(client_id='jupyter', clean_session=True)
client.on_connect = on_connect
client.on_message = on_message
client.username_pw_set("admin", "admin")
client.connect(brokerAddress, port=port, keepalive=60)
print("Broker: %s:%s" % (brokerAddress,port))

client.loop_start()

while Connected is not True:
    time.sleep(0.1)
    
try:
    global topic
    client.subscribe(topic)
    
    for i in range(1000):
        ret = publishRandomTimeSeriesData(client, topic)
        time.sleep(0.5)
    # Blocking call that processes network traffic, dispatches callbacks and handles reconnecting.
    # Other loop*() functions are available that give a threaded interface and a manual interface.
    #client.loop_forever()
  
except KeyboardInterrupt:
    print('disconnect')
    client.disconnect()